## Automatic Differentiation

- Key points about derivatives in deep learning:
  * Essential for optimization algorithms
  * Used in training deep networks
  * Manual calculation is:
    - Tedious
    - Error-prone
    - More difficult with complex models

- Modern deep learning frameworks provide:
  * Automatic differentiation (autograd)
  * Computational graph tracking
  * Backpropagation implementation
    - Works backwards through graph
    - Applies chain rule
    - Efficient gradient computation


In [ ]:
import torch

## A Simple Function

- Goal: Differentiate $y = 2\mathbf{x}^{\top}\mathbf{x}$ with respect to $\mathbf{x}$
- Initial setup:


In [ ]:
x = torch.arange(4.0)
x

- Gradient storage considerations:
  * Need space to store gradients
  * Avoid new memory allocation for each derivative
  * Important because:
    - Deep learning requires many derivative computations
    - Same parameters used repeatedly
    - Memory efficiency crucial
  * Gradient shape matches input vector shape


In [ ]:
x.requires_grad_(True)
x.grad  # The gradient is None by default

--- 

- Function calculation:


In [ ]:
y = 2 * torch.dot(x, x)
y

- Gradient computation:
  * Use `backward()` method
  * Access via `grad` attribute
  * Expected result: $4\mathbf{x}$


In [ ]:
y.backward()
x.grad
x.grad == 4 * x

- Important note about gradient accumulation:
  * PyTorch adds new gradients to existing ones
  * Useful for optimizing sum of multiple objectives
  * Reset with `x.grad.zero_()`


In [ ]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

## Backward for Non-Scalar Variables

- Vector derivatives:
  * Natural interpretation: Jacobian matrix
  * Contains partial derivatives of each component
  * Higher-order tensors for higher-order inputs

- Common use case:
  * Sum gradients of each component
  * Often needed for batch processing
  * Results in vector matching input shape

- PyTorch implementation:
  * Requires explicit reduction to scalar
  * Uses vector $\mathbf{v}$ for computation
  * Computes $\mathbf{v}^\top \partial_{\mathbf{x}} \mathbf{y}$
  * Argument named `gradient` for historical reasons


In [ ]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

## Detaching Computation

- Purpose: Move calculations outside computational graph
- Use cases:
  * Create auxiliary terms without gradients
  * Focus on direct influence of variables
  * Control gradient flow

- Example scenario:
  * `z = x * y` and `y = x * x`
  * Want direct influence of `x` on `z`
  * Solution: Detach `y` to create `u`
  * Results in:
    - Same value as `y`
    - No gradient flow through `u`
    - Direct computation of `z = x * u`


In [ ]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

- Important notes:
  * Detaches ancestors from graph
  * Original graph for `y` persists
  * Can still compute gradients for `y`


In [ ]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

## Gradients and Python Control Flow

- Key feature: Works with dynamic computation paths
- Supports:
  * Conditional statements
  * Loops
  * Arbitrary function calls
  * Variable-dependent control flow

- Example function:

In [ ]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

---

- Implementation details:
  * Graph built during execution
  * Specific path for each input
  * Supports backward pass after execution
  * Works with linear functions and piecewise definitions


In [ ]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a

- Real-world applications:
  * Text processing with variable lengths
  * Dynamic model architectures
  * Statistical modeling
  * Impossible to compute gradients a priori

## Discussion

- Impact of automatic differentiation:
  * Massive productivity boost
  * Enables complex model design
  * Frees practitioners for higher-level tasks

- Technical aspects:
  * Optimization of autograd libraries
  * Compiler and graph manipulation tools
  * Memory efficiency
  * Computational efficiency

- Basic workflow:
  1. Attach gradients to target variables
  2. Record target value computation
  3. Execute backpropagation
  4. Access resulting gradient

## Exercises

1. Backpropagation behavior:
   * Run function twice
   * Observe and explain results

2. Control flow analysis:
   * Change `a` to vector/matrix
   * Analyze non-scalar results
   * Explain computation changes

3. Automatic differentiation practice:
   * Plot $f(x) = \sin(x)$
   * Plot derivative using autograd
   * Avoid using known derivative formula

---

4. Chain rule exercise:
   * Function: $f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$
   * Create dependency graph
   * Compute derivative using chain rule
   * Map terms to dependency graph

5. Let $f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$. Write out a dependency graph tracing results from $x$ to $f(x)$. 

---

6. Use the chain rule to compute the derivative $\frac{df}{dx}$ of the aforementioned function, placing each term on the dependency graph that you constructed previously. 
